In [19]:
import pandas as pd
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import time
import os

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

import csv
import pandas as pd
import configparser

In [2]:
navigator = webdriver.Chrome(ChromeDriverManager().install())
navigator.get(os.getenv('URL'))

/tmp/ipykernel_1231/1960473131.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navigator = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
def click(path, config, type='xpath'):
    if type == 'xpath':
        navigator.find_element(By.XPATH, config['XPaths'][path]).click()
    if type == 'name':
        navigator.find_element(By.NAME, config['XPaths'][path]).click()

In [4]:
def translate_title(title):
    if title == 'Ano':
        return 'YEAR'
    if title == 'Km':
        return 'KM'
    if title == 'Combustível':
        return 'FUEL'  
    if title == 'Cor':
        return 'COLOR'
    if title == 'Estimativa de mercado':
        return 'FIPE'
    if title == 'Pátio':
        return 'YARD'
    if title == 'Retomada':
        return 'RESUME-TYPE'
    if title == 'Tipo':
        return 'TYPE'  
    if title == 'Possui Chave':
        return 'HAS-KEY'

In [5]:
def get_element_text(navigator, config, path):
    value = navigator.find_element(By.XPATH, config['XPaths'][path]).text
    # value = navigator.find_element(By.XPATH, os.getenv(path)).text
    return value

In [6]:
def get_vehicle_data():
    title = []
    value = []
    for i in range(1,10):
        try:     
            result = navigator.find_element(By.XPATH,f"/html/body/div[2]/div/div[1]/main/div[2]/div[2]/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[{i}]/div/div").text.split('\n')
            title.append(result[0])                
            value.append(result[1])
        except:
            pass

    data = {translate_title(title[i]): value[i] for i in range(len(title))}
    return data

##### Get Title

In [28]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [30]:
config['XPaths']['AUCTION-TITLE']

'/html/body/div[2]/div/div[1]/main/div[2]/div[2]/div/div[1]/div/h1'

In [7]:
auction_title = get_element_text(navigator, config, 'AUCTION-TITLE').replace('/','_')
auction_title

'LEILÃO-DE-VEÍCULOS-NORTE-09-03-23'

In [8]:
time.sleep(5)
total_results = int(get_element_text(navigator, config, 'TOTAL-RESULTS').split(' ')[0])
total_results

125

#### Click on car

In [9]:
time.sleep(5)
click('FIRST-CAR')
time.sleep(5)

In [10]:
# click("/html/body/div[2]/div/div[1]/main/div[2]/div[2]/div/div[3]/div/div[1]/div[2]/div[1]/div")
    #    /html/body/div[2]/div/div[1]/main/div[2]/div[2]/div/div[3]/div/div[1]/div[2]/div[1]/div

In [11]:
def write_csv(name, cars):
    with open(f'{name}.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = cars[0].keys())
        writer.writeheader()
        writer.writerows(cars)

In [12]:
def get_information(navigator):
    car_info = get_vehicle_data()
    full_car_info = {'BATCH': get_element_text(navigator, 'BATCH'),
        'VEHICLE': get_element_text(navigator, 'VEHICLE'),
        'BANK':get_element_text(navigator, 'BANK'),
        'KM':'',
        'YEAR': '',
        'FUEL': '',
        'COLOR': '',
        'FIPE': '',
        'YARD': '',
        'RESUME-TYPE': '',
        'TYPE':'',
        'HAS-KEY': ''
        }
    full_car_info.update(car_info)
    return full_car_info

In [13]:
cars = []

for _ in range(1, total_results+1):
    try:
        data = get_information(navigator)
        cars.append(data)
        time.sleep(5)
        click('NEXT_VEHICLE') 
    except:
        print('erro')
        click('NEXT_VEHICLE') 

In [14]:
cars

[{'BATCH': '1',
  'VEHICLE': 'renault/kwid zen 1.0 mecanico (4p)',
  'BANK': 'BANCO SANTANDER S/A',
  'KM': '56.674',
  'YEAR': '20/21',
  'FUEL': 'Flex',
  'COLOR': 'BRANCA',
  'FIPE': 'R$ 50.268,00',
  'YARD': 'PATIO MANAUS',
  'RESUME-TYPE': 'Recuperado de Financiamento',
  'TYPE': 'Veículos',
  'HAS-KEY': 'Sim'},
 {'BATCH': '4',
  'VEHICLE': 'honda/hr-v touring',
  'BANK': 'BANCO SANTANDER S/A',
  'KM': '87.401',
  'YEAR': '18/18',
  'FUEL': 'Flex',
  'COLOR': 'BRANCA',
  'FIPE': 'R$ 107.286,00',
  'YARD': 'BENEVIDES-PA',
  'RESUME-TYPE': 'Recuperado de Financiamento',
  'TYPE': 'Veículos',
  'HAS-KEY': 'Sim'},
 {'BATCH': '5',
  'VEHICLE': 'land rover/evoque se dynamic',
  'BANK': 'BANCO SANTANDER S/A',
  'KM': '62.731',
  'YEAR': '17/17',
  'FUEL': 'Gasolina',
  'COLOR': 'VERMELHA',
  'FIPE': 'R$ 184.987,00',
  'YARD': 'PATIO MANAUS',
  'RESUME-TYPE': 'Recuperado de Financiamento',
  'TYPE': 'Utilitários',
  'HAS-KEY': 'Sim'},
 {'BATCH': '6',
  'VEHICLE': 'renault/kwid zen 2 (4p)'

In [15]:
write_csv(name = auction_title, cars = cars)

In [16]:
vehicles = pd.read_csv('LEILÃO DE VEÍCULOS NORDESTE - 08_03_23.csv')

In [17]:
vehicles.head(2)

,BATCH,VEHICLE,BANK,KM,YEAR,FUEL,COLOR,FIPE,YARD,RESUME-TYPE,TYPE,HAS-KEY
0,1,renault/kwid zen 1.0 mecanico,BANCO SANTANDER S/A,23.368,20/21,Flex,BRANCA,"R$ 49.991,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
1,2,chevrolet/prisma joye 1.0 mecanico (4p),BANCO SANTANDER S/A,58.003,19/19,Flex,BRANCA,"R$ 53.894,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim


In [18]:
vehicles.sort_values(by='KM').head(10)

,BATCH,VEHICLE,BANK,KM,YEAR,FUEL,COLOR,FIPE,YARD,RESUME-TYPE,TYPE,HAS-KEY
66,139,honda/pcx 150 sport abs,BANCO SANTANDER S/A,8.414,20/21,Gasolina,PRATA,"R$ 18.164,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
57,120,honda/cg 160 fan,BANCO SANTANDER S/A,11.884,22/22,Flex,VERMELHA,"R$ 15.772,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
59,125,haojian/avelloz az1,BANCO SANTANDER S/A,11.922,21/22,Gasolina,BRANCA,"R$ 7.499,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
65,137,kawasak/z400,BANCO SANTANDER S/A,16.192,21/21,Gasolina,PRETA,"R$ 28.924,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
0,1,renault/kwid zen 1.0 mecanico,BANCO SANTANDER S/A,23.368,20/21,Flex,BRANCA,"R$ 49.991,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
28,34,volkswagen/gol mc4 (4p),BANCO VOLKSWAGEN S/A,30.498,19/20,Flex,VERMELHA,"R$ 51.823,00",PATIO FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
22,28,volkswagen/polo comfortline ad,BANCO VOLKSWAGEN S/A,50.641,19/20,Flex,AZUL,"R$ 84.521,00",PATIO FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
96,213,ford/fiesta flex (4p),BANCO SANTANDER S/A,51.049,13/14,Flex,BRANCA,"R$ 30.046,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
5,6,fiat/argo drive 1.0 (4p),BANCO SANTANDER S/A,53.201,18/18,Flex,VERMELHA,"R$ 54.644,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
1,2,chevrolet/prisma joye 1.0 mecanico (4p),BANCO SANTANDER S/A,58.003,19/19,Flex,BRANCA,"R$ 53.894,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
